In [33]:
# Import library
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains


import time

from datetime import date

from bs4 import BeautifulSoup
import smtplib

In [35]:
# Set up Selenium
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
#options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(service=service, options=options)


# Go to Google Maps home
driver.get("https://www.google.com/maps")

# Wait for the search box to appear
search_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.ID, "searchboxinput"))
)

# Enter business type and location (for example: "Furniture in Addis Ababa")
business_type = "Furniture"
location = "Nairobi"
query = f"{business_type} in {location}"

# Type and search
search_box.clear()
search_box.send_keys(query)
search_box.send_keys(Keys.ENTER)

# Wait for search results to load
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'm6QErb'))
)


<selenium.webdriver.remote.webelement.WebElement (session="51b0ef6c466491e098d6c60c68d23f48", element="f.8B9B2AB3B9FE8F0C5625DEADABFD2C02.d.8136819C1758FD8851C0CAB29B7AD0FC.e.75")>

In [19]:
# Locate the scrollable div using XPath
scrollable_div = driver.find_element(By.XPATH, '//div[@role="feed"]')

def scroll_page_down():
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    for i in range(30):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
        time.sleep(2)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height


In [21]:
Business_titles = []
Business_ratings = []
Business_maps = []
Business_phones = []
Business_categories = []
Business_addresses = []
Business_opening_hours = []
Business_websites = []

Date_extracted = []


def scrape_page():

    j = 0
    
    selenium = driver.find_elements(By.CLASS_NAME, "Nv2PK")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    Business_List = soup.find_all('div', class_="Nv2PK THOPZb CpccDe")
    print(len(Business_List))
    print(len(selenium))
    
    for i in Business_List:

        # Scrape Business titles
        Business_title = i.find('div', class_="qBF1Pd fontHeadlineSmall")
        if Business_title:
            Business_title = Business_title.get_text(strip=True)
        else:
            Business_title = "N/A"
        Business_titles.append(Business_title)

        # Scrape Business ratings
        Business_rating = i.find("div", class_="W4Efsd").find("div")
        if Business_rating:
            Business_rating = Business_rating .get_text(strip=True)
        else: 
            Business_rating = "N/A"
        Business_ratings.append(Business_rating)

        # Scrape Business webites
        Business_map = i.find('a', class_="hfpxzc")
        if Business_map:
            Business_map = Business_map.get('href')
        else:
            Business_map = "N/A"
        Business_maps.append(Business_map)


        # Scrape Business phones
        Business_phone = i.find('span', class_="UsdlK")
        if Business_phone:
            Business_phone = Business_phone.get_text(strip=True)
        else:
            Business_phone = "N/A"
        Business_phones.append(Business_phone)

        
        # Scrape Business Categories        
        Business_category = i.find_all('div', class_="W4Efsd")
        if Business_category:
            spans = Business_category[1].find_all("span")
            if spans:
                Business_category = spans[0].get_text(strip=True)
            else:
                Business_category = "N/A"
        else:
            Business_category = "N/A"
        Business_categories.append(Business_category)


        # Scrape Business address
        address_found = False
        Business_address = "N/A"
        
        Business_address_divs = i.find_all('div', class_="W4Efsd")
        for div in Business_address_divs:
            spans = div.find_all("span")
            for span in spans:
                text = span.get_text(strip=True)
                # Check for a street keyword (St, Rd, Ave, etc.) — you can extend this
                if any(kw.lower() in text.lower() for kw in ["St", "Street", "City", "Next", "Building", "Bldg", "Town", "Around", "Near", "Road", "Rd", "Ave", "ቦታ", "አድራሻ"]):
                    Business_address = text
                    address_found = True
                    break
            if address_found:
                break
        
        Business_addresses.append(Business_address)

       # Check whether there is a blocking popup before clicking on the business
        try:
            close_btn = driver.find_element(By.CLASS_NAME, 'e2moi')
            close_btn.click()
            WebDriverWait(driver, 5).until_not(
                EC.presence_of_element_located((By.CLASS_NAME, "e2moi"))
            )
        except:
            pass
        
        # Scroll and click using ActionChains
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", selenium[j])
            time.sleep(0.5)
            ActionChains(driver).move_to_element(selenium[j]).click().perform()
        except Exception as e:
            print(f"Could not click on business {j}: {e}")


        print(f"Scraping the {j} th business")
        j = j + 1

        #wait for the business name to show in the side panel
        try:
            WebDriverWait(driver, 10).until(
                EC.text_to_be_present_in_element((By.CLASS_NAME, "DUwDvf"), Business_title)
            )
        except:
            pass  # just a wait, doesn’t return an element

        try:
            website_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.CsEnBe"))
            )
            business_website = website_button.get_attribute("href")
        except:
            business_website = "N/A"
        
        Business_websites.append(business_website)


        
        # Scrape business openening hours
        soup_detail = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup_detail.find('table', class_='eK4R0e')
    
        weekly_hours = []
        
        if table:
            rows = table.find_all('tr', class_='y0skZc')
            
            for row in rows:
                day_cell = row.find('td', class_='ylH6lf')
                day = day_cell.get_text(strip=True) if day_cell else "N/A"
                
                hours_cell = row.find('td', class_='mxowUb')
                hours = hours_cell.get_text(strip=True) if hours_cell else "N/A"
                
                weekly_hours.append(f"{day}: {hours}")
        else:
            weekly_hours.append("N/A")
    
        Business_opening_hours.append(weekly_hours)


        # write the date that the information is extracted 
        Date_extracted.append(date.today())



In [23]:
scroll_page_down()


In [25]:
scrape_page()

# Once done, close the browser
driver.quit()

90
90
Scraping the 0 th business
Scraping the 1 th business
Scraping the 2 th business
Scraping the 3 th business
Scraping the 4 th business
Scraping the 5 th business
Scraping the 6 th business
Scraping the 7 th business
Scraping the 8 th business
Scraping the 9 th business
Scraping the 10 th business
Scraping the 11 th business
Scraping the 12 th business
Scraping the 13 th business
Scraping the 14 th business
Scraping the 15 th business
Scraping the 16 th business
Scraping the 17 th business
Scraping the 18 th business
Scraping the 19 th business
Scraping the 20 th business
Scraping the 21 th business
Scraping the 22 th business
Scraping the 23 th business
Scraping the 24 th business
Scraping the 25 th business
Scraping the 26 th business
Scraping the 27 th business
Scraping the 28 th business
Scraping the 29 th business
Scraping the 30 th business
Scraping the 31 th business
Scraping the 32 th business
Scraping the 33 th business
Scraping the 34 th business
Scraping the 35 th busin

In [27]:
import pandas as pd

df = pd.DataFrame({"Company_name":Business_titles,
                   "Rating":Business_ratings,
                   "Map":Business_maps,
                   "Phpne":Business_phones,
                   "Business_type":Business_categories,
                   "Address":Business_addresses,
                   "Open_time":Business_opening_hours,
                   "Website":Business_websites,
                   "Extraction Date":Date_extracted})

styled_df = df.style.set_properties(**{'text-align': 'left'})
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

In [29]:
styled_df

,Company_name,Rating,Map,Phpne,Business_type,Address,Open_time,Website,Extraction Date
0,Unique Furniture Kenya (Nairobi Branch),4.9(153),https://www.google.com/maps/place/Unique+Furniture+Kenya+%28Nairobi+Branch%29/data=!4m7!3m6!1s0x182f110d0720895d:0x71abf2960262f062!8m2!3d-1.3271256!4d36.8501777!16s%2Fg%2F11b5phv7k8!19sChIJXYkgBw0RLxgRYvBiApbyq3E?authuser=0&hl=am&rclk=1,+254 703 639703,የቤት ዕቃዎች መደብር,"·MRFX+5X5 Kelico Complex, Mombasa Road",['N/A'],N/A,2025-04-26
1,Mufasa Furniture and Deco Ltd,5.0(433),https://www.google.com/maps/place/Mufasa+Furniture+and+Deco+Ltd/data=!4m7!3m6!1s0x182f17b92492f74d:0xfc83a5c7dd070fe0!8m2!3d-1.2772352!4d36.8290566!16s%2Fg%2F11qrs_5wxr!19sChIJTfeSJLkXLxgR4A8H3celg_w?authuser=0&hl=am&rclk=1,+254 722 490643,የቤት ዕቃዎች መደብር,·Ytl Building ( Tigoni Centre) 5th Floor,['N/A'],N/A,2025-04-26
2,Tangerine Furniture,4.5(466),https://www.google.com/maps/place/Tangerine+Furniture/data=!4m7!3m6!1s0x182f172a0b099e43:0xfee1fb363ff66139!8m2!3d-1.3355625!4d36.8964344!16s%2Fg%2F11c1pcc5dz!19sChIJQ54JCyoXLxgROWH2Pzb74f4?authuser=0&hl=am&rclk=1,+254 791 708894,የቤት ዕቃዎች መደብር,N/A,['N/A'],N/A,2025-04-26
3,ODDS & ENDS,"4.5(1,461)",https://www.google.com/maps/place/ODDS+%26+ENDS/data=!4m7!3m6!1s0x182f11ec4df7a811:0x10f6d0395e6f0676!8m2!3d-1.3314453!4d36.8634496!16s%2Fg%2F1pzq_14z_!19sChIJEaj3TewRLxgRdgZvXjnQ9hA?authuser=0&hl=am&rclk=1,+254 737 771717,የቤት ዕቃዎች መደብር,·Mombasa Road,['N/A'],N/A,2025-04-26
4,Victoria Furnitures Ltd,4.3(424),https://www.google.com/maps/place/Victoria+Furnitures+Ltd/data=!4m7!3m6!1s0x182f171506626589:0x24f57d78ef7c71f3!8m2!3d-1.2627815!4d36.8058661!16s%2Fg%2F1pztnmcd6!19sChIJiWViBhUXLxgR83F873h99SQ?authuser=0&hl=am&rclk=1,+254 705 918145,የቢሮ ዕቃዎች መደብር,"·Victoria Courts, Parklands Rd",['N/A'],N/A,2025-04-26
5,Furniture Palace (Int) K Ltd.,4.2(214),https://www.google.com/maps/place/Furniture+Palace+%28Int%29+K+Ltd./data=!4m7!3m6!1s0x182f11ead1d17175:0x75b5ff5b432e720f!8m2!3d-1.3261968!4d36.8485822!16s%2Fg%2F11g9vsyh_f!19sChIJdXHR0eoRLxgRD3IuQ1v_tXU?authuser=0&hl=am&rclk=1,+254 722 201201,የቤት ዕቃዎች መደብር,"·DTB Centre, Mombasa Road",['N/A'],N/A,2025-04-26
6,Naifur designs(Nairobi furniture),4.8(24),https://www.google.com/maps/place/Naifur+designs%28Nairobi+furniture%29/data=!4m7!3m6!1s0x182f6dbffcca1ee7:0x9f8dacc3dad409b1!8m2!3d-1.2865629!4d36.9790609!16s%2Fg%2F11qpm4yl_7!19sChIJ5x7K_L9tLxgRsQnU2sOsjZ8?authuser=0&hl=am&rclk=1,+254 791 374385,የቤት ዕቃዎች መደብር,·Bakhita - Crescent Road,['N/A'],N/A,2025-04-26
7,Furniture Elegance Ltd,4.4(39),https://www.google.com/maps/place/Furniture+Elegance+Ltd/data=!4m7!3m6!1s0x182f1729c27bee91:0xdb9ff6be6c624ffc!8m2!3d-1.276993!4d36.821598!16s%2Fg%2F1thzy83m!19sChIJke57wikXLxgR_E9ibL72n9s?authuser=0&hl=am&rclk=1,+254 702 842500,የቤት ዕቃ አከፋፋይ,·Murang'a Rd,['N/A'],N/A,2025-04-26
8,Royalview Furniture kenya,4.2(5),https://www.google.com/maps/place/Royalview+Furniture+kenya/data=!4m7!3m6!1s0x182f17a2daf5cd13:0x5ad5404fc323e4c7!8m2!3d-1.2777006!4d36.8291119!16s%2Fg%2F11h_wn20pz!19sChIJE8312qIXLxgRx-Qjw09A1Vo?authuser=0&hl=am&rclk=1,+254 726 429320,የቤት ዕቃዎች መደብር,"·Nellea plaza, Ngara Rd","['ቅዳሜ: 7ጥዋት–7ከሰዓት', 'እሑድ: 10ጥዋት–6ከሰዓት', 'ሰኞ: 7ጥዋት–7ከሰዓት', 'ማክሰኞ: 7ጥዋት–7ከሰዓት', 'ረቡዕ: 7ጥዋት–7ከሰዓት', 'ሐሙስ(Labour Day): 7ጥዋት–7ከሰዓትሰዓታት የተለያዩ ሊሆኑ ይችላሉ', 'ዓርብ: 7ጥዋት–7ከሰዓት']",http://www.naifurdesigns.com/,2025-04-26
9,Dignity Furniture Showroom - Langata Branch,4.5(319),https://www.google.com/maps/place/Dignity+Furniture+Showroom+-+Langata+Branch/data=!4m7!3m6!1s0x182f11627b287db5:0xff4663b902fe0cec!8m2!3d-1.3293537!4d36.804747!16s%2Fg%2F11g0tv3jhk!19sChIJtX0oe2IRLxgR7Az-ArljRv8?authuser=0&hl=am&rclk=1,+254 754 666333,የቤት ዕቃዎች መደብር,·Langata Rd,"['ቅዳሜ: 7ጥዋት–7ከሰዓት', 'እሑድ: 10ጥዋት–6ከሰዓት', 'ሰኞ: 7ጥዋት–7ከሰዓት', 'ማክሰኞ: 7ጥዋት–7ከሰዓት', 'ረቡዕ: 7ጥዋት–7ከሰዓት', 'ሐሙስ(Labour Day): 7ጥዋት–7ከሰዓትሰዓታት የተለያዩ ሊሆኑ ይችላሉ', 'ዓርብ: 7ጥዋት–7ከሰዓት']",http://www.naifurdesigns.com/,2025-04-26


In [31]:
df.to_csv("Generated_leads.csv")